In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy import integrate
from scipy import linalg
from scipy import interpolate
from sklearn import gaussian_process as gp
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *
from Adam import Adam
from deeponet import *
from util import *

In [2]:
ntrain = 1000
ntest = 60
curr = generate(samples=2*ntrain,out_dim=51,length_scale=0.2)
N = curr.shape[0]//2
grid = np.linspace(0,1,51)
f = np.zeros((N,grid.shape[0],grid.shape[0]))
f0 = np.zeros((N,grid.shape[0],grid.shape[0]))+1
u = np.zeros((N,grid.shape[0],grid.shape[0]))
u0 = np.zeros((N,grid.shape[0],grid.shape[0]))
for i in range(N):
    f[i] = generate_2(curr[i*2],curr[i*2+1])
    u[i] = FD_2d(f[i],grid)
    if i==0:
        u0[i] = FD_2d(f0[i],grid)
    else:
        u0[i] = u0[0]
    print('\rGenerating u: {:d}/{:d}'.format(i+1,N), end='', flush=True)
    
f_aug = 1*f+10
u_aug = 1*u+10*u0
f_aug = np.concatenate((f,f_aug))
u_aug = np.concatenate((u,u_aug))

curr_1 = generate(samples=2*ntest,out_dim=51,length_scale=0.2)
curr_2 = generate(samples=2*ntest,out_dim=51,length_scale=2)
curr_3 = generate(samples=2*ntest,out_dim=51,length_scale=0.2,A=10)
curr_4 = generate(samples=2*ntest,out_dim=51,length_scale=2,A=10)
curr = np.concatenate((curr_1,curr_2))
curr = np.concatenate((curr,curr_3))
curr = np.concatenate((curr,curr_4))
N = curr.shape[0]//2
f_test = np.zeros((N,grid.shape[0],grid.shape[0]))
u_test = np.zeros((N,grid.shape[0],grid.shape[0]))
for i in range(N):
    f_test[i] = generate_2(curr[i*2],curr[i*2+1])
    u_test[i] = FD_2d(f_test[i],grid)
    print('\rGenerating u: {:d}/{:d}'.format(i+1,N), end='', flush=True)

Generating u: 240/24000

In [3]:
N = f.shape[0]
loc = np.zeros((N,2))
res = np.zeros((N,1))
for i in range(N):
    j = np.random.randint(grid.shape[0],size=2)
    loc[i,:] = j/grid.shape[0]
    res[i,0] = u[i,j[0],j[1]]

In [4]:
f_train = torch.Tensor(f)
f_train = torch.reshape(f_train,(f_train.shape[0],-1))
loc_train = torch.Tensor(loc)
u_train = torch.Tensor(res)    
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_train, loc_train, u_train), batch_size=100, shuffle=True)

N = f_aug.shape[0]
loc = np.zeros((N,2))
res = np.zeros((N,1))
for i in range(N):
    j = np.random.randint(grid.shape[0],size=2)
    loc[i,:] = j/grid.shape[0]
    res[i,0] = u_aug[i,j[0],j[1]]

f_aug = torch.Tensor(f_aug)
f_aug = torch.reshape(f_aug,(f_aug.shape[0],-1))
loc_aug = torch.Tensor(loc)
u_aug = torch.Tensor(res)
aug_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_aug, loc_aug, u_aug), batch_size=100, shuffle=True)

dim = f_test.shape[-1]
N = f_test.shape[0]*dim*dim
loc = np.zeros((N,2))
res = np.zeros((N,1))
f = np.zeros((N,dim,dim))
j = 0
for i in range(N):
    f[i] = f_test[i//(dim**2)]
    if j//dim>=dim:
        j = 0
    loc[i,0] = grid[j//dim]
    loc[i,1] = grid[i%dim]
    res[i,0] = u_test[i//(dim**2),j//dim,i%dim]
    j += 1
f_test = torch.Tensor(f)
f_test = torch.reshape(f_test,(f_test.shape[0],-1))
loc = torch.Tensor(loc)
res = torch.Tensor(res)
test_1_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_test[:ntest*dim*dim], loc[:ntest*dim*dim], res[:ntest*dim*dim]), batch_size=dim*dim, shuffle=False)
test_2_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_test[ntest*dim*dim:ntest*2*dim*dim], loc[ntest*dim*dim:ntest*dim*dim*2], res[ntest*dim*dim:ntest*2*dim*dim]), batch_size=dim*dim, shuffle=False)
test_3_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_test[ntest*2*dim*dim:ntest*3*dim*dim], loc[ntest*2*dim*dim:ntest*3*dim*dim], res[ntest*2*dim*dim:ntest*3*dim*dim]), batch_size=dim*dim, shuffle=False)
test_4_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(f_test[ntest*3*dim*dim:], loc[ntest*3*dim*dim:], res[ntest*3*dim*dim:]), batch_size=dim*dim, shuffle=False)


In [5]:
naug = 2000
batch_size = 100
learning_rate = 0.001
epochs = 600
step_size = 50
gamma = 0.5

model = DeepONet(dim*dim,2).cuda()
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
for ep in range(epochs):
    model.train()
    train_mse = 0
    for x, l, y in train_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(x,l)
        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')
        mse.backward()
        optimizer.step()
        train_mse += mse.item()
    scheduler.step()
    train_mse /= len(train_loader)
    print('\repoch {:d}/{:d} MSE = {:.10f}'.format(ep+1,epochs,train_mse), end='', flush=True)
    
print('\n')
myloss = LpLoss(size_average=False)
test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_1_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()
    test_mse /= len(test_1_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)


test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_2_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()        
    test_mse /= len(test_2_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)
    

test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_3_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()        
    test_mse /= len(test_3_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)
    
test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_4_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()        
    test_mse /= len(test_4_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)
    
torch.save(model,'model/deeponet2_1')

epoch 600/600 MSE = 0.0000506494

test error: L2 =  0.3596180422852437 MSE = 0.00015201339532116737
test error: L2 =  0.29160404180487 MSE = 4.199769876625699e-05
test error: L2 =  0.13830239127079647 MSE = 0.003233601147076115
test error: L2 =  0.13555942525466283 MSE = 0.003020395249283562


In [6]:
model = DeepONet(dim*dim,2).cuda()
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

for ep in range(epochs):
    model.train()
    train_mse = 0
    for x, l, y in aug_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(x,l)
        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')
        mse.backward()
        optimizer.step()
        train_mse += mse.item()
    scheduler.step()
    train_mse /= len(aug_loader)
    print('\repoch {:d}/{:d} MSE = {:.10f}'.format(ep+1,epochs,train_mse), end='', flush=True)
    
print('\n')
test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_1_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()
    test_mse /= len(test_1_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)

test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_2_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()
    test_mse /= len(test_2_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)
    
test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_3_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()        
    test_mse /= len(test_3_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)
    

test_mse = 0
test_l2 = 0
with torch.no_grad():
    for x, l, y in test_4_loader:
        x, l, y = x.cuda(), l.cuda(), y.cuda()
        out = model(x,l).view(-1)
        mse = F.mse_loss(out.view(1, -1), y.view(1, -1), reduction='mean')
        l2 = myloss(out.view(1, -1), y.view(1, -1))
        test_mse += mse.item()
        test_l2 += l2.item()        
    test_mse /= len(test_4_loader)
    test_l2 /= ntest
    print('test error: L2 = ',test_l2,'MSE =',test_mse)

epoch 600/600 MSE = 0.0001676839

test error: L2 =  0.38426088268558184 MSE = 0.00017586410000755375
test error: L2 =  0.2323009792715311 MSE = 4.038107163069071e-05
test error: L2 =  0.0782296701023976 MSE = 0.0010368791389434287
test error: L2 =  0.07419719956815243 MSE = 0.0009048418617264057


In [7]:
torch.save(model,'model/deeponet2_2')